# Align Macroeconomic Indicators to daily frequency

## Import Packages

In [1]:
import os
from pathlib import Path
import logging
import warnings

import pandas as pd
pd.set_option('future.no_silent_downcasting', True)

import matplotlib.pyplot as plt
import numpy as np
import ruptures as rpt

from datetime import datetime
import exchange_calendars as xc

# for Parquet I/O
import pyarrow as pa
import pyarrow.parquet as pq

## Read in Files

In [2]:
ffill_files = Path("../../../02_Data_Processed/02_Macroeconomic_Indicators/01_Forward_Filled/_fileNames.txt").read_text(encoding="utf-8").splitlines()
interp_files = Path("../../../02_Data_Processed/02_Macroeconomic_Indicators/02_Interpolated/_fileNames.txt").read_text(encoding="utf-8").splitlines()

## Align to daily frequency with forward fill

In [3]:

src_dir = "../../../02_Data_Processed/02_Macroeconomic_Indicators/01_Forward_Filled"
des_dir = "../../../02_Data_Processed/02_Macroeconomic_Indicators/03_Forward_Filled_Daily"
Path(des_dir).mkdir(parents=True, exist_ok=True)

def forward_fill_align_daily(relative_path, filename):
    file_path = Path(relative_path) / filename
    df = pd.read_parquet(file_path)
    df = df.set_index('date')
    df.index = pd.DatetimeIndex(df.index)
    start, end = df.index.min(), df.index.max()
    all_days = pd.date_range(start, end, freq="D")

    original_dates = set(df.index)
    df_daily = df.reindex(all_days).ffill()
    df_daily = df_daily.infer_objects(copy=False)
    
    df_daily['is_filled_daily'] = ~df_daily.index.isin(original_dates)
    df_daily = df_daily.reset_index().rename(columns={'index': 'date'})
    return df_daily

In [4]:
for f in ffill_files:
    df_daily = forward_fill_align_daily(src_dir, f)
    outname = f.replace('.parquet', '_daily.parquet')
    df_daily.to_parquet(Path(des_dir) / outname, index=False)


## Align to daily frequency with interpolation

In [5]:
src_dir = "../../../02_Data_Processed/02_Macroeconomic_Indicators/02_Interpolated"
des_dir = "../../../02_Data_Processed/02_Macroeconomic_Indicators/04_Interpolated_Daily"
Path(des_dir).mkdir(parents=True, exist_ok=True)


def interpolate_align_daily(relative_path, filename):
    file_path = Path(relative_path) / filename
    df = pd.read_parquet(file_path)
    df = df.set_index('date')
    df.index = pd.DatetimeIndex(df.index)
    start, end = df.index.min(), df.index.max()
    all_days = pd.date_range(start, end, freq="D")

    original_dates = set(df.index)

    df_daily = df.reindex(all_days)
    df_daily = df_daily.infer_objects(copy=False)
    # Interpolate only the value column(s)
    df_daily['value'] = df_daily['value'].interpolate(method='linear', limit_direction='both').round(1)
    df_daily['value_filled'] = df_daily['value_filled'].ffill()
    # Add indicator for interpolated rows
    df_daily['is_interpolated_daily'] = ~df_daily.index.isin(original_dates)
    df_daily = df_daily.reset_index().rename(columns={'index': 'date'})
    return df_daily

In [6]:
for f in interp_files:
    df_daily = interpolate_align_daily(src_dir, f)
    outname = f.replace('.parquet', '_daily.parquet')
    df_daily.to_parquet(Path(des_dir) / outname, index=False)